In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation
from sklearn.metrics import mean_squared_error
import pickle
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("green_tripdata_2015-09.csv")

### Question 4 part 1:
Based on http://www.nyc.gov/html/tlc/html/passenger/taxicab_rate.shtml, the initial charge is $2.50.
So all the records with Fare_amount < 2.5 are removed.

In [22]:
df_tip = df.copy()
df_tip =df_tip[df_tip.loc[:, "Fare_amount"]>2.5]
df_tip.loc[:, "tip%"] = df_tip["Tip_amount"].div(df_tip.Fare_amount)
print "Summary: Tip percentage\n",df_tip["tip%"].describe()

Summary: Tip percentage
count    1.475973e+06
mean     9.288832e-02
std      1.750175e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      2.133333e-01
max      3.750000e+01
Name: tip%, dtype: float64


### Question 4 part 2:
Building a predictive model for tip as a percentage of the total fare.

I used two sources for building the prediction model:
http://www.skytree.net/2016/07/14/nyc-taxi-blog-series-2-of-2-dataset-analysis/
also I got some ideas from this paper:
https://cseweb.ucsd.edu/~jmcauley/cse190/reports/sp15/050.pdf
 

##### First Step for building a model is data cleaning.
Data cleaning consists:
1. removing (Ehail_fee), since since 99% are Null
2. replacing missing and invalid values with the most frequent values categorical variables
3. replacing invalid values with the median for continuous variables
4. replacing negative values for varibales such as fare_amount, tip_amount... with their absolute values 
5. converting datetime varibales to their correct format
6. removing those transaction that "payment_type" is not with Credit Card

##### Secod Step is feature engineering:
1. Add 9 new derived features that are created from pickup and dropoff timestamps, trip distance.
2. Also adding trip_percentage feature

##### Third Step is building a model:
1. I use random forest regressor for building a model(unfortunately due to time limit, I was not able to test other models and compare different models )
2. In order of optimization I optimized the number of trees, since they have significant impact on model accuracy.
3. And number of trees = 80 is the optimized one for a model with important features.
4. I build models with sample size of 100,000 and the models are crossed validate with 3 folds cross-validation
5. The builded models show that important featues are: "Fare_amount", "Total_amount", "Tip_amount" and "trip_duration", "speed_mph"
6. Finally, I used all the samples for buiding the prediction model but since it has a big size I used 100,000 samples for submission. Also, I used only important features and n_estimator = 80
7. Created model saved with pickle in a file named: 'pre_tip.pkl'
8. For making prediction run tip_prediction.make_prediction(test_data)
9. As conclusion, I think by having some information about passenger can affect accuracy as well.
10. Average mean_sqyared_error on different test sample set is: 20.18
#### Running the prediction model
A python file (tip_prediction.py) is provided. For making prediction use tip_prediction.predict(data),  where data is any 2015 raw dataframe from given url: http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml


In [4]:
# *********************** first step is data cleaning ***********************
def data_cleanining(df_new):
    
    # 1. remove "Ehail_fee" column since 99% are Null(missing values)
    del df_new["Ehail_fee"]

    # 2. replace missing value in "Trip_type" by most frequent value which is 1
    mfv_t = df_new['Trip_type '].value_counts().idxmax()
    df_new['Trip_type '].replace(np.NaN, 1,inplace=True)

    # 3.replace invalid value in "RateCodeID" which is 99 by most frequent value which is 1
    # Rate Code values include: 1, 2, 3, 4, 5, 6
    mfv_r = df_new['RateCodeID'].value_counts().idxmax()
    df_new.RateCodeID[~((df_new.RateCodeID>=1) & (df_new.RateCodeID<=6))] = mfv_r

    # 4. replace invalid values in "Extra" by most frequent value which is 0
    # Extra can have only these values: 0, 0.50 and 1
    mfv_e = df_new['Extra'].value_counts().idxmax()
    df_new.Extra[~((df_new.Extra==0.5) |(df_new.Extra==1) | (df_new.RateCodeID==0.5))] = mfv_e

    # 5. replace invalid values in "Passenger_count" which is 0 by most frequent value which is 1
    mfv_p = df_new['Passenger_count'].value_counts().idxmax()
    df_new.Passenger_count[~((df_new.Passenger_count> 0))] = mfv_p

    # 6. replace values in "Trip_distance" <=0 by median
    med_td = df_new['Trip_distance'].median()
    df_new.Trip_distance[(df_new.Trip_distance <= 0)] = med_td

    # 7. Negative values found and replaced by their abs
    df_new.Fare_amount = df_new.Fare_amount.abs()
    df_new.Tip_amount = df_new.Tip_amount.abs()
    df_new.Total_amount = df_new.Total_amount.abs()
    df_new.Tolls_amount = df_new.Tolls_amount.abs()
    df_new.improvement_surcharge = df_new.improvement_surcharge.abs()
    df_new.MTA_tax = df_new.MTA_tax.abs()

    # 8. replace values in "Fare_amount" <  2.5 by median
    med_fm= df_new['Fare_amount'].median()
    df_new.Fare_amount[(df_new.Fare_amount < 2.5)] = med_fm

    # 9. replace values in "Total_amount" <  2.5 by median
    med_tm= df_new['Total_amount'].median()
    df_new.Total_amount[(df_new.Total_amount < 2.5)] = med_tm

    # 10. Assuming 0 = N and 1 = Y, change N to 0 and Y to 1
    df_new['Store_and_fwd_flag'].replace('Y', 1, inplace=True)
    df_new['Store_and_fwd_flag'].replace('N', 0, inplace=True)

    # 11. converting the "lpep_pickup_datetime" and "Lpep_dropoff_datetime" to DateTime series
    df_new.loc[:, "lpep_pickup_datetime"] = pd.to_datetime(df_new.loc[:, "lpep_pickup_datetime"])
    df_new.loc[:, "Lpep_dropoff_datetime"] = pd.to_datetime(df_new.loc[:, "Lpep_dropoff_datetime"])

    # 12. remove those rows that "payment_type" is not with Credit Card (1)
    df_new = df_new[df_new.Payment_type == 1]
    return df_new

In [5]:
# *********************** Second step is feature engineering ***********************
def data_engin(df_new):
    
    # 1. creating time varibales from "lpep_pickup_datetime" and "Lpep_dropoff_datetime"

    # int [0-23], hour the day a transaction was done
    df_new.loc[:, "hour_p"] = df_new.loc[:, "lpep_pickup_datetime"].dt.hour
    df_new.loc[:, "hour_d"] = df_new.loc[:, "Lpep_dropoff_datetime"].dt.hour
    # int [0-6], day of the week a transaction was done
    df_new.loc[:, "weekday_p"] = df_new.loc[:, "lpep_pickup_datetime"].dt.weekday
    df_new.loc[:, "weekday_d"] = df_new.loc[:, "Lpep_dropoff_datetime"].dt.weekday
    #Month_day: int [0-30], day of the month a transaction was done
    df_new.loc[:, "monthday_p"] = df_new.loc[:, "lpep_pickup_datetime"].dt.day
    df_new.loc[:, "monthday_d"] = df_new.loc[:, "Lpep_dropoff_datetime"].dt.day
    # Trip Duration in minutes
    df_new.loc[:,'trip_duration'] = df_new.loc[:, "Lpep_dropoff_datetime"] - df_new.loc[:, "lpep_pickup_datetime"]
    df_new.loc[:,'trip_duration'] = df_new.loc[:,'trip_duration'].dt.total_seconds()/60
    # removing "lpep_pickup_datetime" and "Lpep_dropoff_datetime"
    del df_new["lpep_pickup_datetime"]
    del df_new["Lpep_dropoff_datetime"]


    # 2. speed variable
    df_new.loc[:,'speed_mph'] = df_new.Trip_distance/(df_new.trip_duration/60)
    # remove transactions that speed in not in valid range
    df_new = df_new[((df_new.speed_mph>0) & (df_new.speed_mph<=240))]
    # 3. location variable

    # 4.  create tip percentage variable
    df_new['tip_percentage'] = 100*df_new.Tip_amount/df_new.Fare_amount
    return df_new

In [6]:
# this function finds the optimized number of estimators of the model

def find_optimized_number_est(data_sample):
    """
        this function finds the optimized number of estimators of the model
        data_sample: pandas data frame
        return integer as optimized # of estimator
    """
    data = data_sample.ix[:, data_sample.columns != 'tip_percentage']
    target = data_sample['tip_percentage']
    # randomly hold out 20% of the data as test set
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    data, target, test_size=.20, random_state=0)
    mse_final = []
    for j in range(50, 220, 20):
        rf = RandomForestRegressor(n_estimators=j)
        rf.fit(X_train, y_train)
        predicted = rf.predict(X_test)
        mse = mean_squared_error(y_test, predicted)
        mse_final.append((j, mse))
    min_mse = min(mse_final, key = lambda t: t[1])
    print(min_mse[0])
    return min_mse[0]

In [7]:
def evaluate_prediction_with_RFR(df_sample, n_opt, k):
    
    """
        Build and Evaluate the model using k-fold cross-validation
        param data: pandas dataframe
        param n_opt: optimised number of estimator
        param k: k_fold CV
        return Mean Squared error with 3 fold cross_validation

    """
    data = df_sample.ix[:, df_sample.columns != 'tip_percentage']
    target = df_sample['tip_percentage']
    mse_avg = []
    for i in range(k): # repeat the procedure k times to get more precise results
        
        # for each iteration, randomly hold out 20% of the data as test set
        X_train, X_test, y_train, y_test = cross_validation.train_test_split(
        data, target, test_size=.20, random_state=0)
        rf = RandomForestRegressor(n_estimators= n_opt)
        rf.fit(X_train, y_train)
    #         print(rf.feature_importances_)
        predicted = rf.predict(X_test)
        print("score", rf.score(X_test, y_test))
        mse = mean_squared_error(y_test, predicted)
        print("Mean Squared error", mse)
        mse_avg.append(mse)
    return np.mean(mse_avg)

In [8]:
def save_optimized_prediction_model(df_sample, n_opt):
    """
    build the prediction model with all the samples and save 
    the model with pickle 
    """
#     print(len(df_sample.columns))
    data = df_sample.ix[:, df_sample.columns != 'tip_percentage']
    target = df_sample['tip_percentage']
    rf = RandomForestRegressor(n_estimators= n_opt)
    rf.fit(data, target)
#     print(rf.feature_importances_)
    # save the model to disk
    filename = 'pre_tip.pkl'
    pickle.dump(rf, open(filename, 'wb'))
#     return 

In [9]:
def predict_tip(test):
    """
        this function loads the model from disk and predict the tip for test data
        param test: pandas data frame that is cleaned and feature engineering is done
    """
    # load the model from disk
    filename = 'pre_tip.pkl'
    loaded_model = pickle.load(open(filename, 'rb'))
    X_test = test.ix[:, test.columns != 'tip_percentage']
    Y_test = test['tip_percentage']
    predicted = loaded_model.predict(X_test)
    print("score", loaded_model.score(X_test, Y_test ))
    mse = mean_squared_error(Y_test , predicted)
    print("Mean Squared error", mse)
    return predicted

### optimization and validation
In this section I found optimized number of estimators, which is 150. (for a model with all the features) 

I build a model with sample size of 100,000 and evaulate the model using k-fold cross-validation (k=3).

In [9]:
df_new = df.copy()
print "data cleaning"
df_new = data_cleanining(df_new)
print "feature engineering"
df_new = data_engin(df_new)
# # find optimized n_estimators on sample data of size 100,000
df_sample = df_new.loc[np.random.choice(df_new.index,size=100000,replace=False)]
n_opt = find_optimized_number_est(df_sample)
n_opt = 150
# evaluating the model with 3 folds cross-validation
k = 3 # number of CV
# Sample size for training and optimization was chosen as 100,000 with 3 folds cross-validation
df_sample = df_new.loc[np.random.choice(df_new.index,size=100000,replace=False)]
# Build and Evaluate the model using k-fold cross-validation
mse_final = evaluate_prediction_with_RFR(df_sample, n_opt, k)
print ("Mean Squared error with", k, "fold cross_validation" , mse_final)

data cleaning
feature engineering
('score', 0.95884104640622525)
('Mean Squared error', 37.094230075193245)
('score', 0.96388886669833063)
('Mean Squared error', 32.544915990543892)
('score', 0.95743067148872574)
('Mean Squared error', 38.365320982857014)
('Mean Squared error with', 3, 'fold cross_validation', 36.00148901619805)


### build and save model
This section is for saving the best model for later use

In [53]:
# build and save model for later use
df_new = df.copy()
print "data cleaning"
df_new = data_cleanining(df_new)
print "feature engineering"
df_new = data_engin(df_new)
df_sample = df_new.loc[np.random.choice(df_new.index,size=100000,replace=False)]
# # df_sample = df_new # build the model with all the sample
# print "buiding the model with all sample as save with pickl"
att = ["Fare_amount", "Total_amount", "Tip_amount", "trip_duration", "speed_mph", "tip_percentage"]
save_optimized_prediction_model(df_sample[att], 80)

data cleaning
feature engineering
[ 0.38738767  0.05085928  0.53513401  0.00736876  0.01925027]


### load the model and predict tip for test

In [13]:
# load the model and predict tip for test
df_new = df.copy()
print "data cleaning"
df_new = data_cleanining(df_new)
print "feature engineering"
df_new = data_engin(df_new)
test = df_new.loc[np.random.choice(df_new.index,size=100000,replace=False)]
att = ["Fare_amount", "Total_amount", "Tip_amount", "trip_duration", "speed_mph", "tip_percentage"]
predict_tip(test[att])

('score', 0.98090748740972156)
('Mean Squared error', 26.762485045646425)


array([ 28.64      ,  22.5       ,   0.        , ...,  20.9122807 ,
        13.79310345,  27.92988148])